### Data Preprocessing

In [1]:
import pandas as pd
import cbrkit
import polars as pl
import numpy as np
import ranx  # for evaluation


## site_path ='https://www.kaggle.com/datasets/yasserhessein/multiclass-diabetes-dataset'
### original_data = 'https://data.mendeley.com/datasets/wj9rwkp9c2/1'

data = pd.read_csv('/Users/macbookair/Downloads/Multiclass Diabetes Dataset/Multiclass Diabetes Dataset.csv')

In [2]:
#read the dataset
df = pd.read_csv('/Users/macbookair/Downloads/Dataset of Diabetes .csv')

In [3]:
# view the top 5 rows
df.head()

,ID,No_Pation,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI,CLASS
0,502,17975,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
1,735,34221,M,26,4.5,62,4.9,3.7,1.4,1.1,2.1,0.6,23.0,N
2,420,47975,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
3,680,87656,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
4,504,34223,M,33,7.1,46,4.9,4.9,1.0,0.8,2.0,0.4,21.0,N


In [4]:
#shape of the dataset
df.shape

(1000, 14)

In [5]:
# view the number of duplicated ID
df['ID'].duplicated().sum()

200

In [6]:
# view the check for duplicated IDs
df[df['ID'].duplicated(keep=False)].sort_values(by='ID')

,ID,No_Pation,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI,CLASS
846,1,34325,M,56,20.8,800,9.0,4.6,2.0,1.2,2.5,0.9,35.0,Y
273,1,34325,M,58,20.8,800,9.1,6.6,2.9,1.1,4.3,1.3,33.0,Y
847,2,44835,M,60,2.1,56,7.2,6.3,3.7,1.0,3.7,1.7,28.0,Y
541,2,44835,M,60,2.1,56,7.6,3.3,1.7,0.9,1.7,0.8,36.6,Y
848,3,41248,M,53,4.2,55,8.0,5.0,2.5,1.3,2.3,1.6,30.0,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,198,34288,M,39,4.3,49,5.1,6.7,2.5,1.1,4.5,1.1,25.0,Y
994,199,454316,M,69,10.3,185,7.7,4.9,1.9,1.2,3.0,0.7,37.0,Y
753,199,48613,F,68,4.2,41,11.9,4.0,1.1,2.3,1.1,0.5,30.0,Y
241,200,24038,M,57,2.0,77,7.0,5.5,1.9,1.0,3.7,0.9,33.0,Y


In [7]:
## drop  duplicated ID I believe the later values are more recent entries
df = df.drop_duplicates(subset='ID', keep='last')

In [8]:
# view the number of categorical classes
df['CLASS'].unique()

array(['N', 'N ', 'P', 'Y', 'Y '], dtype=object)

In [9]:
#count of each class
df['CLASS'].value_counts()

CLASS
Y     703
N      57
P      35
Y       4
N       1
Name: count, dtype: int64

In [10]:
## replace the error of the categorical values
df['CLASS'] = df['CLASS'].replace({'N ': "N", "Y ": "Y"})

## view the count of each class after replacement
print(df['CLASS'].value_counts())

CLASS
Y    707
N     58
P     35
Name: count, dtype: int64


In [11]:
# file_path ='/Users/macbookair/Documents/UFSM/COM804/diabetes_cleaned.csv'

In [12]:
## read the dataset
# data = pd.read_csv('/Users/macbookair/Documents/UFSM/COM804/diabetes_cleaned.csv')
#print top 5 rows
data = df.copy()
data.head()

,ID,No_Pation,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI,CLASS
0,502,17975,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
1,735,34221,M,26,4.5,62,4.9,3.7,1.4,1.1,2.1,0.6,23.0,N
2,420,47975,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
3,680,87656,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
4,504,34223,M,33,7.1,46,4.9,4.9,1.0,0.8,2.0,0.4,21.0,N


In [13]:
data.describe()

,ID,No_Pation,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI
count,800.0000,8.000000e+02,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,400.5000,3.237742e+05,53.880000,5.064554,67.558750,8.499325,4.955900,2.331387,1.224563,2.621050,1.934750,29.923062
std,231.0844,3.777967e+06,8.364542,2.892686,57.591015,2.468488,1.311315,1.397187,0.706753,1.167342,3.930872,4.877135
min,1.0000,1.230000e+02,20.000000,0.500000,20.000000,0.900000,0.600000,0.300000,0.200000,0.300000,0.100000,19.000000
25%,200.7500,2.408775e+04,51.000000,3.600000,47.000000,6.800000,4.100000,1.500000,0.900000,1.700000,0.700000,26.000000
50%,400.5000,3.441250e+04,55.000000,4.500000,59.000000,8.200000,4.800000,2.000000,1.100000,2.500000,1.000000,30.000000
75%,600.2500,4.538850e+04,59.000000,5.700000,73.000000,10.200000,5.700000,2.900000,1.350000,3.300000,1.500000,33.000000
max,800.0000,7.543566e+07,79.000000,38.900000,800.000000,16.000000,10.300000,13.800000,9.900000,9.900000,35.000000,47.750000


There are huge outliers in attributes like Cr, VLDL and a litle in the rest. so we need to clean the outliers

In [14]:
# Extract the numeric columns and drop the ID, No_Pation and AGE columns
num_cols = data.select_dtypes(include=["number"]).columns.drop(['ID', 'No_Pation', 'AGE'])
print(num_cols)

Index(['Urea', 'Cr', 'HbA1c', 'Chol', 'TG', 'HDL', 'LDL', 'VLDL', 'BMI'], dtype='object')


In [15]:
## remove outliers 
Q3 = data[num_cols].quantile(0.75)
Q1 = data[num_cols].quantile(0.25)
IQR = Q3 - Q1
hi = Q3 + 1.5 * IQR
lo = Q1 - 1.5 * IQR 

#filters he dataset to find ehre the columns that has a lower or higher value than the IQR
outlier_mask = (data[num_cols] < lo) | (data[num_cols] > hi)
##selects rows that has outliers
data_outliers = data[outlier_mask.any(axis=1)]
##drops rows that has outliers and reset the index
data_clean = data.loc[~outlier_mask.any(axis=1)].copy().reset_index(drop=True)

In [16]:
data_clean.describe()

,ID,No_Pation,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI
count,607.000000,6.070000e+02,607.000000,607.000000,607.000000,607.000000,607.000000,607.000000,607.000000,607.000000,607.000000,607.000000
mean,390.123558,8.805579e+04,53.754530,4.493081,57.861614,8.431565,4.858155,2.126524,1.119160,2.614234,0.994893,29.452059
std,228.878842,5.124807e+05,8.335421,1.468431,17.355646,2.481233,1.130201,1.005384,0.321607,1.027955,0.466845,4.692790
min,2.000000,1.230000e+02,20.000000,0.500000,20.000000,3.700000,2.000000,0.300000,0.400000,0.750000,0.100000,19.000000
25%,201.500000,2.408100e+04,51.000000,3.400000,45.000000,6.750000,4.100000,1.400000,0.900000,1.800000,0.700000,26.000000
50%,387.000000,3.439000e+04,55.000000,4.400000,56.000000,8.200000,4.900000,2.000000,1.100000,2.500000,0.900000,30.000000
75%,569.500000,4.351400e+04,59.000000,5.400000,70.000000,10.200000,5.600000,2.700000,1.300000,3.400000,1.300000,33.000000
max,800.000000,8.785782e+06,79.000000,8.800000,106.000000,15.000000,8.100000,5.000000,2.000000,5.600000,2.600000,39.000000


In [17]:
data_clean.head()

,ID,No_Pation,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI,CLASS
0,735,34221,M,26,4.5,62,4.9,3.7,1.4,1.1,2.1,0.6,23.0,N
1,504,34223,M,33,7.1,46,4.9,4.9,1.0,0.8,2.0,0.4,21.0,N
2,634,34224,F,45,2.3,24,4.0,2.9,1.0,1.0,1.5,0.4,21.0,N
3,721,34225,F,50,2.0,50,4.0,3.6,1.3,0.9,2.1,0.6,24.0,N
4,421,34227,M,48,4.7,47,4.0,2.9,0.8,0.9,1.6,0.4,24.0,N


In [18]:
# # convert the cleaned dataframe from Pandas DF to Polars DF
# new_data = pl.from_pandas(df)

In [19]:
data_clean.CLASS.value_counts()

CLASS
Y    527
N     45
P     35
Name: count, dtype: int64

In [20]:
## Split the dataset
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data_clean, test_size=0.2, random_state=42, stratify=data_clean['CLASS'])

X_train = train_data.drop('CLASS', axis=1)
y_train = train_data['CLASS']

X_test = test_data.drop('CLASS', axis=1)
y_test = test_data['CLASS']


In [21]:
X_train.describe()

,ID,No_Pation,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI
count,485.000000,4.850000e+02,485.000000,485.000000,485.000000,485.000000,485.000000,485.000000,485.000000,485.000000,485.000000,485.000000
mean,388.428866,8.230690e+04,53.868041,4.510309,58.185567,8.460536,4.846804,2.146598,1.105835,2.602247,1.009691,29.359443
std,228.140855,4.588046e+05,8.186054,1.480234,17.394980,2.486449,1.142789,1.012960,0.312879,1.029522,0.475526,4.700414
min,2.000000,2.150000e+02,26.000000,0.500000,23.000000,3.700000,2.000000,0.300000,0.400000,0.750000,0.100000,19.000000
25%,204.000000,2.407500e+04,51.000000,3.400000,46.000000,6.800000,4.100000,1.400000,0.900000,1.800000,0.700000,26.000000
50%,383.000000,3.439800e+04,55.000000,4.300000,56.000000,8.200000,4.800000,2.000000,1.100000,2.500000,0.900000,30.000000
75%,567.000000,4.443300e+04,59.000000,5.400000,70.000000,10.200000,5.600000,2.700000,1.300000,3.400000,1.300000,33.000000
max,798.000000,8.785782e+06,79.000000,8.800000,106.000000,15.000000,8.000000,5.000000,2.000000,5.600000,2.600000,39.000000


In [22]:
## load the data into a casebase
casebase = cbrkit.loaders.pandas(train_data)
casebase

pandas(df=      ID  No_Pation Gender  AGE  Urea  Cr  HbA1c  Chol   TG  HDL  LDL  VLDL  \
51   393      47496      M   49   5.0  74    6.2   2.0  0.8  0.6  1.0   0.4   
346  352      35706      F   55   2.9  41    9.5   5.9  1.7  1.2  3.6   0.8   
374  586        856      F   61   4.3  56   12.1   4.4  2.0  1.0  2.5   0.9   
156  296      24070      F   56   5.5  48    9.3   5.8  2.4  0.9  3.9   1.1   
71   705      87668      M   33   4.8  64    5.8   4.8  1.1  1.7  2.6   0.5   
..   ...        ...    ...  ...   ...  ..    ...   ...  ...  ...  ...   ...   
354  246      48817      M   56   6.3  87   12.4   5.0  2.3  0.8  3.2   1.0   
159  484       3451      F   56   5.5  48    9.3   5.8  2.4  0.9  3.9   1.1   
79   715      87673      M   50   4.7  53    6.1   4.2  2.2  0.8  2.5   0.9   
277  512      20098      M   51   4.9  56    7.7   3.0  0.3  1.3  1.5   0.1   
546   85      41741      F   44   4.6  54    9.4   6.8  2.9  1.1  4.5   1.3   

      BMI CLASS  
51   25.0     P  
346  

In [23]:
## try multiple queries 
# ten_data = data_clean[:10]
mul_queries = pl.from_pandas(X_test.drop(columns = ['ID', 'No_Pation']))

queries = mul_queries.to_dicts()
queries[9]

{'Gender': 'M',
 'AGE': 33,
 'Urea': 2.0,
 'Cr': 54,
 'HbA1c': 5.4,
 'Chol': 3.7,
 'TG': 1.3,
 'HDL': 0.8,
 'LDL': 2.4,
 'VLDL': 0.6,
 'BMI': 22.0}

In [24]:
test_data

,ID,No_Pation,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI,CLASS
250,757,34387,M,60,3.0,55,7.7,4.2,4.9,1.6,1.7,1.0,33.0,Y
588,159,45446,M,54,5.0,88,6.8,6.7,3.7,0.9,2.9,1.7,32.0,Y
218,434,24116,F,63,2.4,66,7.4,7.1,4.1,1.5,3.8,1.0,38.0,Y
180,353,24089,M,55,4.6,77,6.5,6.0,1.5,1.3,4.1,0.6,30.0,Y
331,550,123,F,51,4.2,72,11.2,4.4,2.1,1.1,2.5,0.9,26.0,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,458,34242,F,39,3.2,55,5.0,4.6,1.3,1.0,3.0,0.6,21.0,N
528,62,34545,M,59,4.6,53,13.3,5.4,3.0,1.2,3.0,1.3,29.0,Y
7,402,34235,F,50,3.5,39,4.0,4.0,1.5,1.2,2.2,0.7,24.0,N
439,471,34534,F,58,3.7,36,10.7,4.2,0.5,0.9,3.1,0.2,36.0,Y


In [25]:
## Define Query

#convert test data to query
test_query = pl.from_pandas(test_data)
# queries = cbrkit.loaders.polars(test_query)
query = test_query.to_dicts()[0]
query

{'ID': 757,
 'No_Pation': 34387,
 'Gender': 'M',
 'AGE': 60,
 'Urea': 3.0,
 'Cr': 55,
 'HbA1c': 7.7,
 'Chol': 4.2,
 'TG': 4.9,
 'HDL': 1.6,
 'LDL': 1.7,
 'VLDL': 1.0,
 'BMI': 33.0,
 'CLASS': 'Y'}

In [26]:
# #### Create similiarity search

# mac_sim = cbrkit.sim.attribute_value(
#     attributes = {
#         "Gender": cbrkit.sim.generic.equality(),
#     }
# )

# fac_sim = cbrkit.sim.attribute_value(
#     attributes= {
#         "AGE":  cbrkit.sim.numbers.linear(max_distance= 20),
#         "Urea": cbrkit.sim.numbers.linear(max_distance= 1.5),
#         "Cr":   cbrkit.sim.numbers.linear(max_distance= 17.5),
#         "HbA1c":cbrkit.sim.numbers.linear(max_distance= 2.5),
#         "Chol": cbrkit.sim.numbers.linear(max_distance= 1.5),
#         "TG":   cbrkit.sim.numbers.linear(max_distance= 1.0),
#         "HDL":  cbrkit.sim.numbers.linear(max_distance=0.33),
#         "LDL":  cbrkit.sim.numbers.linear(max_distance=1.05),
#         "VLDL": cbrkit.sim.numbers.linear(max_distance=0.5),
#         "BMI":  cbrkit.sim.numbers.linear(max_distance= 5)
#     }
# )


# mac_retriever = cbrkit.retrieval.build(mac_sim, minimum_similiarity=1.0)
# fac_retriever = cbrkit.retrieval.build(fac_sim, limit=10)

# result = cbrkit.retrieval.apply(casebase, query, [mac_retriever, fac_retriever])

In [27]:
#### Create similiarity search

mac_sim = cbrkit.sim.attribute_value(
    attributes = {
        'Gender' : cbrkit.sim.generic.equality()
    }
)

fac_sim = cbrkit.sim.attribute_value(
    attributes= {
        "AGE":  cbrkit.sim.numbers.linear(max=9),
        "Urea": cbrkit.sim.numbers.linear(max=1.5),
        "Cr":   cbrkit.sim.numbers.linear(max=17.5),
        "HbA1c":cbrkit.sim.numbers.linear(max=2.5),
        "Chol": cbrkit.sim.numbers.linear(max=1.5),
        "TG":   cbrkit.sim.numbers.linear(max=1.0),
        "HDL":  cbrkit.sim.numbers.linear(max=0.33),
        "LDL":  cbrkit.sim.numbers.linear(max=1.05),
        "VLDL": cbrkit.sim.numbers.linear(max=0.5),
        "BMI":  cbrkit.sim.numbers.linear(max=5)
    }
)

# Retrieval function to select the top 100 cases where there is an exact similiarity of the same gender  
mac_retriever = cbrkit.retrieval.dropout(cbrkit.retrieval.build(mac_sim),
                                         min_similarity = 1.0,
                                         limit = 400)

##  Function to select the top 10 similar case to the query
fac_retriever = cbrkit.retrieval.dropout(cbrkit.retrieval.build(fac_sim), 
                                        limit=5)


## Retrieval

In [28]:
retrievers = (mac_retriever, fac_retriever)

In [29]:
def run_retriever(casebase, queries, retrievers):
    for q in queries:
        result = cbrkit.retrieval.apply_query(casebase, q, retrievers)
        print(f"\n Results for Query {q}")
        print(result.final_step.ranking)
        print(result.final_step.similarities)

In [30]:
run_retriever(casebase, queries, retrievers)


 Results for Query {'Gender': 'M', 'AGE': 60, 'Urea': 3.0, 'Cr': 55, 'HbA1c': 7.7, 'Chol': 4.2, 'TG': 4.9, 'HDL': 1.6, 'LDL': 1.7, 'VLDL': 1.0, 'BMI': 33.0}
(217, 21, 453, 468, 400)
{217: AttributeValueSim(value=0.5765194805194804, attributes={'AGE': 1.0, 'Urea': 0.8000000000000002, 'Cr': 0.7714285714285715, 'HbA1c': 0.9599999999999997, 'Chol': 0.5333333333333332, 'TG': 0.0, 'HDL': 0.09090909090909081, 'LDL': 0.8095238095238095, 'VLDL': 0.3999999999999999, 'BMI': 0.4}), 21: AttributeValueSim(value=0.5718095238095238, attributes={'AGE': 1.0, 'Urea': 0.4000000000000001, 'Cr': 0.6571428571428571, 'HbA1c': 0.4800000000000001, 'Chol': 0.8666666666666666, 'TG': 0.0, 'HDL': 0.0, 'LDL': 0.7142857142857143, 'VLDL': 1.0, 'BMI': 0.6}), 453: AttributeValueSim(value=0.5678730158730159, attributes={'AGE': 0.7777777777777778, 'Urea': 0.3333333333333333, 'Cr': 1.0, 'HbA1c': 0.9199999999999999, 'Chol': 0.5333333333333332, 'TG': 0.0, 'HDL': 0.0, 'LDL': 0.7142857142857143, 'VLDL': 0.7999999999999998, 'B

### REUSE

##### Reuse for small query

In [31]:
# Adapt each case = leave it unchanged
def adapt_identity(case: dict, query: dict) -> dict:
    return case

# Your existing sims become retrievers
mac_reuse = cbrkit.retrieval.build(mac_sim)
fac_reuse = cbrkit.retrieval.build(fac_sim)
reuse_retrievers = (mac_reuse, fac_reuse)

# Build the reuser (pick ONE adaptation_func from above)
reuser = cbrkit.reuse.build(
    retriever_func = reuse_retrievers,
    adaptation_func = adapt_identity,      # or adapt_identity_map
    multiprocessing = False
)


In [32]:
import numpy as np
from collections import defaultdict

def predict_class_weighted_vote(result, casebase, target_col="CLASS", top_k=None):
    step = getattr(result, "final_step", result)
    ranking = step.ranking  # list of (key, similarity)

    if top_k is not None:
        ranking = ranking[:top_k]
    if not ranking:
        return None

    total = sum(sim for _, sim in ranking) or 1.0
    weights = defaultdict(float)
    for key, sim in ranking:
        w = sim / total
        cls = casebase[key][target_col]
        weights[cls] += w

    # tie-break: pick the class with the highest weight, then by name for stability
    best_weight = max(weights.values())
    tied = [c for c, w in weights.items() if np.isclose(w, best_weight)]
    return sorted(tied)[0]


In [33]:
def 

SyntaxError: invalid syntax (1885169875.py, line 1)

In [ ]:
mac_reuse = cbrkit.retrieval.build(mac_sim)
fac_reuse = cbrkit.retrieval.build(fac_sim)
reuse_retrievers = (mac_reuse, fac_reuse)

reuser = cbrkit.reuse.build(
    retriever_func = reuse_retrievers,
    adaptation_func = 
)

In [ ]:
def identity_adaptation(solution, query, retrieved_case):
    # Solution may be a scalar label or a dict; just return as-is
    return solution

reuse_fn = cbrkit.reuse.build(
    retriever_func=cbrkit.retrieval.build(retrievers),  # run both retrievers
    adaptation_func=identity_adaptation
)


In [ ]:
def predict_class(y_train, queries, retrieved_case):
    return

In [ ]:
mac_reuse = cbrkit.reuse.build(retriever_func = mac_sim  )
fac_reuse = cbrkit.reuse.build(fac_sim)
# mac_retriever = cbrkit.retrieval.dropout(
#     cbrkit.retrieval.build(mac_sim), 
#     min_similarity=1.0, 
#     limit = 400)
# fac_retriever = cbrkit.retrieval.dropout(
#     cbrkit.retrieval.build(fac_sim), 
#     limit=10)

reuse_result = cbrkit.reuse.apply_query(casebase, new_query, retrievers)
    

In [ ]:
mac_reuse = cbrkit.reuse.build(
    retriever_func=cbrkit.retrieval.build(mac_sim),
    solution_attr="CLASS",
    aggregator="majority",
    k=5
)

fac_reuse = cbrkit.reuse.build(
    retriever_func=cbrkit.retrieval.build(fac_sim),
    solution_attr="CLASS",
    aggregator="majority",
    k=5
)
